### Importar Librerias

In [2]:
# Librerias a utilizar en el EDA

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from pandas_datareader import data as pdr
import yfinance as yfin
import datetime as dt
from datetime import datetime

# Librería de la FRED
from fredapi import Fred

# API Key
fred_key = '2e3cf97d1b456831253eda002ce25948'

### Asignar Variables Globales

In [3]:
# Se utiliza un rango de 50 años para la extraccion de datos económicos
timeframe = 365*50
today = datetime.today()
end = today.strftime("%Y-%m-%d")
start = (today - dt.timedelta(days=timeframe)).strftime("%Y-%m-%d")

# Se utiliza un rango de 25 años para la extraccion de datos bursátiles
stock_timeframe = 365*25
stock_start = (today - dt.timedelta(days=stock_timeframe)).strftime("%Y-%m-%d")
stock_end = end

### Extraccion de Datos

#### Extraccion de Datos Economicos de FRED

In [4]:
# Crear el objeto FRED a partir de la API Key
fred = Fred(api_key=fred_key)

In [5]:
# Este diccionario contiene los codigos de series de FRED (key) que se utilizaran para extraer los indicadores (value)
fred_series_dict = {
    'DGS10': '10-Year Treasury Yield', # 10-Year Treasury Rate
    'DGS2': '2-Year Treasury Yield', # 2-Year Treasury Rate
    'DFF': 'Federal Funds Effective Rate', # Federal Funds Effective Rate
    'DGS1MO': '1-Month Treasury Yield', # 1-Month Treasury Rate
    'DGS3MO': '3-Month Treasury Yield', # 3-Month Treasury Rate
    'DGS5': '5-Year Treasury Yield', # 5-Year Treasury Rate
    'DGS20': '20-Year Treasury Yield', # 20-Year Treasury Rate
    'DGS30': '30-Year Treasury Yield', # 30-Year Treasury Rate
    'MORTGAGE15US': '15-Year Mortgage Rate', # 15-Year Mortgage Rate
    'MORTGAGE30US': '30-Year Mortgage Rate', # 30-Year Mortgage Rate
    'UNRATE': 'Unemployment Rate',
    'GDP': 'GDP',
    'A191RP1Q027SBEA': 'GDP Growth',
    'GDPC1': 'Real GDP',
    'A191RL1Q225SBEA': 'RGDP Growth',
    'CPIAUCSL': 'CPI', # Inflation rate: infrate = pd.DataFrame(CPI.pct_change(12),columns=["Inflation Rate"]) # CPI YoY
    'PPIFIS': 'PPI', # PPI rate: ppi = pd.DataFrame(ppi_fd.pct_change(),columns=["PPI"])
    'GFDEBTN': 'Government Debt', # Gov Debt / GDP: govdebt_gdp = pd.DataFrame((govdebt*.1/gdp).dropna(),columns=["Gov Debt to GDP"]) # Gov Debt / GDP
    'DTWEXBGS': 'US Dollar Index',   
}

# Este diccionario se utiliza para crear las columnas que contienen calculos
series_calc_dict = {
    'CPIAUCSL': 'Inflation rate', # Inflation rate: infrate = pd.DataFrame(CPI.pct_change(12),columns=["Inflation Rate"]) # CPI YoY
    'PPIFIS': 'PPI rate', # PPI rate: ppi = pd.DataFrame(ppi_fd.pct_change(),columns=["PPI"])
    'GFDEBTN': 'Government Debt to GDP', # Gov Debt / GDP: govdebt_gdp = pd.DataFrame((govdebt*.1/gdp).dropna(),columns=["Gov Debt to GDP"]) # Gov Debt / GDP
}

# Este diccionario contiene los codigos de serie de la FRED para los tipos de interes (pruebas)
# int_rates_dict = {
#     'DGS1MO': '1-Month Treasury Yield', # 1-Month Treasury Rate
#     'DGS3MO': '3-Month Treasury Yield', # 3-Month Treasury Rate
#     'DGS2': '2-Year Treasury Yield', # 2-Year Treasury Rate
#     'DGS5': '5-Year Treasury Yield', # 5-Year Treasury Rate
#     'DGS10': '10-Year Treasury Yield', # 10-Year Treasury Rate
#     'DGS20': '20-Year Treasury Yield', # 20-Year Treasury Rate
#     'DGS30': '30-Year Treasury Yield' # 30-Year Treasury Rate
# }

In [6]:
indicators_df = pd.DataFrame() # DataFrame para almacenar los datos extraidos

# Acceder al diccionario de indicadores economicos para extraer sus datos (utilizando el objeto 'fred' y la funcion 'get_series') y almacenarlos en un DataFrame
for code,indicator in fred_series_dict.items():
    indicators_df[indicator] = fred.get_series(code)
    # Printamos el resultado de la extraccion de cada indicador con su codigo, indicador, rows y fecha de inicio
    print(f'Code: {code} | Indicator: {indicator} | # Rows: {len(indicators_df[indicator].dropna())} | Start Date: {str(indicators_df[indicator].index[0]).split(" ")[0]}')
    
    # Realizar los siguientes calculos dependiendo del indicador y añadirlo al DataFrame
    if code == 'CES0500000003':
        print(f'-> {code} - {series_calc_dict[code]}')
        indicators_df[series_calc_dict[code]] = (indicators_df[indicator] - indicators_df[indicator].shift(12))/indicators_df[indicator]*100
    elif code == 'CPIAUCSL':
        print(f'-> {code} - {series_calc_dict[code]}')
        indicators_df[series_calc_dict[code]] = indicators_df[indicator].pct_change()
    elif code == 'PPIFIS':
        print(f'-> {code} - {series_calc_dict[code]}')
        indicators_df[series_calc_dict[code]] = indicators_df[indicator].pct_change()
    elif code == 'GFDEBTN':
        print(f'-> {code} - {series_calc_dict[code]}')
        indicators_df[series_calc_dict[code]] = (indicators_df[indicator] * .1) / indicators_df['GDP']
    elif code == 'BOPTIMP':
        print(f'-> {code} - {series_calc_dict[code]}')
        indicators_df[series_calc_dict[code]] = indicators_df[indicator] - indicators_df['Exports']
    elif code == 'IEABC':
        print(f'-> {code} - {series_calc_dict[code]}')
        indicators_df[series_calc_dict[code]] = indicators_df[indicator] / indicators_df["GDP"]

Code: DGS10 | Indicator: 10-Year Treasury Yield | # Rows: 15303 | Start Date: 1962-01-02
Code: DGS2 | Indicator: 2-Year Treasury Yield | # Rows: 11711 | Start Date: 1962-01-02
Code: DFF | Indicator: Federal Funds Effective Rate | # Rows: 15985 | Start Date: 1962-01-02
Code: DGS1MO | Indicator: 1-Month Treasury Yield | # Rows: 5425 | Start Date: 1962-01-02
Code: DGS3MO | Indicator: 3-Month Treasury Yield | # Rows: 10401 | Start Date: 1962-01-02
Code: DGS5 | Indicator: 5-Year Treasury Yield | # Rows: 15303 | Start Date: 1962-01-02
Code: DGS20 | Indicator: 20-Year Treasury Yield | # Rows: 13614 | Start Date: 1962-01-02
Code: DGS30 | Indicator: 30-Year Treasury Yield | # Rows: 11533 | Start Date: 1962-01-02
Code: MORTGAGE15US | Indicator: 15-Year Mortgage Rate | # Rows: 1648 | Start Date: 1962-01-02
Code: MORTGAGE30US | Indicator: 30-Year Mortgage Rate | # Rows: 2713 | Start Date: 1962-01-02
Code: UNRATE | Indicator: Unemployment Rate | # Rows: 525 | Start Date: 1962-01-02
Code: GDP | Indi

In [7]:
# Crear nuevas columnas para los 'spread' de los tipos de interes (tipos a largo plazo - tipos a cortos plazo)
# 3m5y,3m10y, 2y10y, 2y20y, 5y10y, 10y30y, 10yTrea30yFRM

indicators_df["3m5y"] =  indicators_df["5-Year Treasury Yield"] - indicators_df["3-Month Treasury Yield"]
indicators_df["3m10y"] = indicators_df["10-Year Treasury Yield"] - indicators_df["3-Month Treasury Yield"]
indicators_df["2y10y"] = indicators_df["10-Year Treasury Yield"] - indicators_df["2-Year Treasury Yield"]
indicators_df["2y20y"] = indicators_df["20-Year Treasury Yield"] - indicators_df["2-Year Treasury Yield"]
indicators_df["5y10y"] = indicators_df["10-Year Treasury Yield"] - indicators_df["5-Year Treasury Yield"]
indicators_df["10y30y"] = indicators_df["30-Year Treasury Yield"] - indicators_df["10-Year Treasury Yield"]
indicators_df["10yTrea30yFRM"] = indicators_df["30-Year Mortgage Rate"] - indicators_df["10-Year Treasury Yield"]

In [16]:
# Guardar el DataFrame como un archivo csv
indicators_df.to_csv(r"C:\Users\Joan Oliver\TheBridge-DataScienceBootcamp\EDA\src\data\raw\indicators_df.csv")

#### Extraccion de Datos de Mercado de Valores (US Stock Market Data)

In [34]:
# Permite crear el DataFrame
yfin.pdr_override()

# Crear la lista de los diferentes mercados de valores y sectores
stocks = ['SPY','DIA','^IXIC','XLE','XLF','XLU','XLI','XLK','XLV','XLY','XLP','XLB','IYR','GDX']

# Extraer los precios de !Yahoo Finanzas para cada uno de los indices y almacenarlos en el DataFrame 'markets'
markets = pdr.get_data_yahoo(stocks,start=start,end=end)

# Filtrar el DataFrame quedandonos con la columna de 'Adj Close' y el rango temporal previamente definido
market_hist = markets["Adj Close"].loc[stock_start:stock_end]

[*********************100%***********************]  14 of 14 completed


In [35]:
# Guardar el DataFrame como un archivo csv
market_hist.to_csv(r"C:\Users\Joan Oliver\TheBridge-DataScienceBootcamp\EDA\src\data\raw\market_hist.csv")

In [32]:
indicators_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15985 entries, 1962-01-02 to 2023-04-10
Data columns (total 29 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   10-Year Treasury Yield        15303 non-null  float64
 1   2-Year Treasury Yield         11711 non-null  float64
 2   Federal Funds Effective Rate  15985 non-null  float64
 3   1-Month Treasury Yield        5425 non-null   float64
 4   3-Month Treasury Yield        10401 non-null  float64
 5   5-Year Treasury Yield         15303 non-null  float64
 6   20-Year Treasury Yield        13614 non-null  float64
 7   30-Year Treasury Yield        11533 non-null  float64
 8   15-Year Mortgage Rate         1648 non-null   float64
 9   30-Year Mortgage Rate         2713 non-null   float64
 10  Unemployment Rate             525 non-null    float64
 11  GDP                           175 non-null    float64
 12  GDP Growth                    175 non-null 

In [33]:
market_hist.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6286 entries, 1998-04-20 00:00:00-04:00 to 2023-04-11 00:00:00-04:00
Data columns (total 13 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   DIA     6286 non-null   float64
 1   IYR     5739 non-null   float64
 2   SPY     6286 non-null   float64
 3   XLB     6114 non-null   float64
 4   XLE     6114 non-null   float64
 5   XLF     6114 non-null   float64
 6   XLI     6114 non-null   float64
 7   XLK     6114 non-null   float64
 8   XLP     6114 non-null   float64
 9   XLU     6114 non-null   float64
 10  XLV     6114 non-null   float64
 11  XLY     6114 non-null   float64
 12  ^IXIC   6286 non-null   float64
dtypes: float64(13)
memory usage: 687.5 KB
